In [1]:
import tweepy
import tweepy.auth
import datetime
import codecs
import csv
import json

In [2]:
with open("../credential.json") as fp:
    credentials = json.load(fp)

In [6]:
#python で Twitter APIを使用するためのConsumerキー、アクセストークン設定
# Consumer_key = '(ご自身でキーを取得する必要があります)'
# Consumer_secret = '(ご自身でキーを取得する必要があります)'
# Access_token = '(ご自身でキーを取得する必要があります)'
# Access_secret = '(ご自身でキーを取得する必要があります)'

#認証

class BearerAuth(tweepy.auth.AuthHandler):
    def __init__(self, bearer):
        self.bearer = bearer

    def apply_auth(self, _url=None, _method=None, _headers=None, _parameters=None):
        return tweepy.auth.OAuth2Bearer(self.bearer)

    def get_username(self):
        raise NotImplemented

In [7]:
# auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret)
# auth.set_access_token(Access_token, Access_secret)

auth = BearerAuth(credentials["twitter"]["bearer_token"])
api = tweepy.API(auth, wait_on_rate_limit = True)

In [8]:
#検索キーワード設定 
# q = "(#movewhatmatters -RT) OR (#MoveWhatMatters -RT)"
q = "早稲田大学 OR #早稲田 OR #waseda OR 早稲田 OR ワセダ -RT"
#q = "テレワーク拡大も6割が｢従業員のストレス増加｣企業はメンタルケアが課題 OR 新型コロナウイルス感染症への組織対応に関する研究調査 OR 新型コロナウイルス感染症への組織対応に関する緊急調査 -RT"

#つぶやきを格納するリスト
tweets_data =[]

#tweepy.Cursour を用いてデータを取得する
#for文で、tweet にデータを放り込んでいく
#q に検索するキーワードが指定されている
for tweet in tweepy.Cursor(api.search, q=q, count=3000,tweet_mode='extended').items():

    #ツイート時間を日本時間に変換する
    jsttime = tweet.created_at + datetime.timedelta(hours=9)
    #print(jsttime)
    #print(tweets_data)

    #つぶやきテキスト(FULL)を取得
    #tweets_data.append(tweet.full_text + '\n')
    
    #リツイートやFAV 数を取得する
    #tweet.id -> ツイートのID
    #tweet.user -> ユーザ名
    #tweet.location -> ツイートをした場所
    #tweet.created_at -> ツイートをした時間
    #tweet.full_text -> ツイート内容を取得する
    #tweet.favorite_count -> ファボ回数を取得する
    #tweet.retweet_count -> リツイート回数を取得する
    #それらをtweets_dataに保存していく
    tweets_data.append([tweet.id,tweet.user.screen_name,tweet.user.location,tweet.created_at,tweet.full_text.replace('\n',''), tweet.favorite_count,tweet.retweet_count])

In [13]:
print(len(tweets_data))
for tweet in tweets_data[:5]:
    print(tweet)
print("...")
for tweet in tweets_data[-5:]:
    print(tweet)

18641
[1440950537156009984, 'ba17_Eri', '大崎', datetime.datetime(2021, 9, 23, 8, 5, 54), '対面授業週0の女子大生が降臨した (@ 早稲田大学 戸山キャンパス - @waseda_univ in Shinjuku, 東京都) https://t.co/SxGZ7JsmJP', 0, 0]
[1440950412308201477, 'waibotwi', '日吉かな？四谷かな？', datetime.datetime(2021, 9, 23, 8, 5, 24), 'また、一部の熱狂的早稲田教信徒は、全学部を受験するというエクストリーム・スポーツに挑戦する。', 0, 0]
[1440950126197936134, 'y10_1m', '東京', datetime.datetime(2021, 9, 23, 8, 4, 16), '【※応募殺到】偏差値は４０台だけど本気で一流大学に合格したい人限定。（早稲田、慶応、国立旧帝国大学）偏差値を２０以上アップし一流大学に合格することを約束した勉強法を無料でプレゼント中。https://t.co/9NcsAS1Uq0', 0, 0]
[1440950083403485189, 'JFl0523WUC', '', datetime.datetime(2021, 9, 23, 8, 4, 6), 'それにしてもアナウンサーにならなくて良かったww 早稲田のアナ研にいたら絶対どこかの局アナになれてたと思うけど、そしたら天才的な数々の曲は生まれなかったことになるからなあ。酒井さんを誘った村上さん、流石だわ。', 0, 0]
[1440950079234342918, 'kokobekn', '東京', datetime.datetime(2021, 9, 23, 8, 4, 5), '【※応募殺到】偏差値は４０台だけど本気で一流大学に合格したい人限定。（早稲田、慶応、国立旧帝国大学）偏差値を２０以上アップし一流大学に合格することを約束した勉強法を無料でプレゼント中。https://t.co/vbbEcCmnXa', 0, 0]
...
[1437776508228186114, 'shibunsawaeiich', '北海道パズル全国1位(暫定)

In [14]:
#ファイル出力
#wordcloud 用
#with codecs.open('./toyota.txt', 'w','utf8') as f:
#    f.writelines(tweets_data)

#f.close()

#ファイル出力
with open('./waseda.csv', 'w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    #tweets_data を書き込んでいく
    writer.writerow(["id","user","location","created_at","text","fav","RT"])
    writer.writerows(tweets_data)
pass